# Learning sentence representations from natural language inference data

##### University of Amsterdam, ATCS: Practical I - April 2023

## Introduction

This is an attempt to reproduce the results from the "Supervised Learning of Universal Sentence Representations from Natural Language Inference Data"  (https://arxiv.org/abs/1705.02364). 
It uses [SentEval](https://github.com/facebookresearch/SentEval) to automatically evaluate the quality of the sentence representations on a wide range of tasks. 

## Results

## Original Paper
(https://arxiv.org/abs/1705.02364)
| **Model**  |         | **NLI** |          | **Transfer** |           |
|------------|---------|:-------:|----------|:------------:|-----------|
|            | **dim** | **dev** | **test** | **micro**    | **macro** |
| Baseline   | 300     | -       | -        | -            | -         |
| LSTM       | 2048    | 81.9    | 80.7     | 79.5         | 78.6      |
| BiLSTM-Last| 4096    | -       | -        | -            | -         |
| BiLSTM-Max | 4096    | 85      | 84.5     | 85.2         | 83.7      |

## Reproduction implementation

| **Model**  |         | **NLI** |          | **Transfer** |           |
|------------|---------|:-------:|----------|:------------:|-----------|
|            | **dim** | **dev** | **test** | **micro**    | **macro** |
| Baseline   | 300     | 65.3    | 65.3     | 78.3         | 76.7      |
| LSTM       | 2048    | 79.3    | 80.0     |   82.8       |  81.0     |
| BiLSTM-Last| 4096    | 80.9    | 80.6     | 78.8         | 77.2      |
| BiLSTM-Max | 4096    | 83.5    | 83.5     |   83.6       |    82.0   |

#### Discussion

All models have been reproduced well within margins (+/- 3%) of the original paper, therefore we can continue with the analysis of the results. We can clearly see that the baseline model underperforms significantly compared to the other models. Because it naively averages the word embedding of each individual word, the sentence representation possibly loses information about the sequential structure between words and their context within the sentence. The LSTM encoder significantly does better than the baseline on all tasks, clearly indicating that the sequential nature is important to represent sentences. This is further confirmed by the additional improvement on the NLI task by the BiLSTM encoder, which not only processes information from left to right but also from right to left. Lastly, we can see that the BiLSTM+Max Pooling clearly outperforms all previous encoders, as was also found in the original paper. While the BiLSTM is just a concatenation of the forward and backward pass of the LSTM model, the BiLSTM-Max looks at every individual output value and collects the maximum value across every representation dimension, as a result it possibly places more emphasis on important embeddings found anywhere in the sentence. 

One important thing to note is that the accuracy significantly increases for the Baseline model in the transfer tasks from the NLI task in comparison to the other more advances models. This can be explained by the difference in nature of the extra tasks. While the SNLI task relies much more in sentence structure for accurate prediction, this is much less the case for the more simple tasks present in the SentEval evaluation setting. Another interesting result is that while the BiLSTM encoder outperforms the LSTM encoder on the NLI task, this is not the case for the SentEval setting. As mentioned in the paper, one hypothesis is that this model over-specializes and adapts well to the biases of the dataset without generating general-purpose sentence representations useful for the other tasks. The addition of the Max Pooling feature does outperform the LSTM variant, likely due to its ability to make sharp decisions and focusing on important embedding features of the output.

# Demonstration

In this section we give a demonstration of the NLI Model with the different implemented encoder types.

### Load Models

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

baseline = torch.load("models/BaselineEncoder_model.pt", map_location=device)
lstm = torch.load("models/LSTMEncoder_model.pt", map_location=device)
bilstm = torch.load("models/BiLSTMEncoder_model.pt", map_location=device)
bilstmmax = torch.load("models/BiLSTMMaxPoolEncoder_model.pt", map_location=device)

In [3]:
def predict_models(premise, hypothesis):
    print("Baseline Encoder prediction:       ", baseline.predict(premise, hypothesis))
    print("LSTM Encoder prediction:           ", lstm.predict(premise, hypothesis))
    print("BiLSTM-Last Encoder prediction:    ", bilstm.predict(premise, hypothesis))
    print("BiLSTM+MaxPool Encoder prediction: ", bilstmmax.predict(premise, hypothesis))

### Example 1

In [4]:
premise = "Two men sitting in the sun"
hypothesis = "Nobody is sitting in the shade"
# Label - Neutral (likely predicts contradiction)

predict_models(premise, hypothesis)

Baseline Encoder prediction:        Neutral
LSTM Encoder prediction:            Neutral
BiLSTM Encoder prediction:          Neutral
BiLSTM+MaxPool Encoder prediction:  Neutral


Although the trained models predict the correct label, this might also be predicted as a contradiction. This could possibly be due to the similar sentence structure while at the same time having the heavily contradicting nature of "two men" and "nobody" in addition to "the sun" and "the shade". If we look at the first parts "Two men sitting" and "Nobody is sitting" this sounds contradictory as well as "sitting in the sun" and "sitting in the shade". However, the model apparently can detect that these statements are neutral. Possibly due to the double negative or detection of the different spatial locations they are referring to.

### Example 2

In [5]:
premise = "A man is walking a dog"
hypothesis = "No cat is outside"
# Label - Neutral (likely predicts contradiction)

predict_models(premise, hypothesis)

Baseline Encoder prediction:        Neutral
LSTM Encoder prediction:            Neutral
BiLSTM Encoder prediction:          Neutral
BiLSTM+MaxPool Encoder prediction:  Neutral


Although the trained models predict the correct label, this might also be predicted as a contradiction. This could again be due to the contradictory nature between cat and dog, or the first sentence starting with a noun and the second sentence starting with "No" + noun, also indicating a contradiction.

# Further Research

## Experiment 1
This section gives is aimed to demonstrate the strengths of some of the aspects of the trained models with respect to their more simple counterpart, demonstrated by additional examples.

### Baseline vs LSTM

In [8]:
premise = "Two women are embracing while holding to go packages."
hypothesis = "Two woman are holding packages"

print("Baseline Encoder prediction:       ", baseline.predict(premise, hypothesis))
print("LSTM Encoder prediction:           ", lstm.predict(premise, hypothesis))

Baseline Encoder prediction:        Neutral
LSTM Encoder prediction:            Entailment


Here we can clearly see that the Baseline fails where the LSTM is able to predict the correct label of entailment.
An LSTM encoder is able to capture the sequential nature of the sentence, including the relationship between words and their context, which can help it understand the meaning of the sentence more deeply for a longer sentence. On the other hand, the baseline encoder only looks at the context of each word and combines it into a single embedding without processing the sequential structure. As a result, it may struggle to capture the subtle nuances of the language and might not be able to infer more complex sentence structures in order to make an accurate prediction about the hypothesis based on the premise.

### LSTM vs BiLSTM-Last

In [3]:
premise = "The doctor told the patient to take the medicine with food."
hypothesis = "The patient took the medicine without eating."

print("LSTM Encoder prediction:  ", lstm.predict(premise, hypothesis))
print("BiLSTM-Last Encoder prediction:", bilstm.predict(premise, hypothesis))

LSTM Encoder prediction:   Entailment
BiLSTM Encoder prediction: Contradiction


In this example, a BiLSTM encoder has an advantage over a unidirectional LSTM encoder because it can capture the information from the premise in both forward and backward directions, allowing it to better understand the context of the sentence. Specifically, the phrase "with food" suggests that the medicine should be taken alongside a meal or snack, which is relevant to the hypothesis that the patient took the medicine without eating. The BiLSTM can capture this relationship by processing the sentence in both directions and taking into account the full context of the premise.

### BiLSTM-Last vs BiLSTM + Max Pool

In [ ]:
premise = "John's favorite hobby is playing the guitar and singing."
hypothesis = "John has a passion for music."

print("BiLSTM-Last prediction:               ", bilstm.predict(premise, hypothesis))
print("BiLSTM-Max prediction:           ", bilstmmax.predict(premise, hypothesis))

BiLSTM prediction:                Contradiction
BiLSTM-Max prediction:            Entailment


: 

In this example, a BiLSTM encoder with max pooling can better capture the key information in the premise that supports the hypothesis. The phrase "playing the guitar and singing" indicates that John is interested in music, and the use of max pooling can help the model focus on this information and disregard less relevant details. By capturing the most important information in the premise, the model can make a more accurate prediction of the hypothesis.

In contrast, a standard BiLSTM encoder would have to rely on the entire premise sequence, including potentially irrelevant or distracting details, to make its prediction. This can result in the model missing the key information that supports the hypothesis.

## Curriculum Learning

Curriculum learning (CL) is a powerful technique that has proven effective in improving the performance of NLP models. This approach involves training models on a curriculum, gradually increasing the complexity of the tasks they are asked to perform.

The way complexity is defined in this setting is by the total sentence length (number of tokens in the premise + number of tokens in the entailment). This assumes that longer sentences are harder to predict entailment for than for shorter sentences.

The hypothesis is that by starting with simpler examples and gradually increasing the difficulty, the model can gradually learn to handle more complex examples and avoid getting stuck in suboptimal local minima during training. Another way that curriculum learning can lead to faster convergence by reducing the amount of noise in the training data. By presenting the model with cleaner and more structured training examples first, it can learn to better discriminate between relevant and irrelevant information in the input, and reduce the impact of noisy or irrelevant information later on. This can help prevent the model from overfitting to the noise in the training data and can lead to better generalization performance. Lastly, it could potentially speed up the training process by reducing computation as the batches will contain sentences of similar length, requiring less padding leading to smaller input tensors. However the difference could be limited as this is potentially already handled by the torch.utils.nn.pack_padded_sequence.

#### Results
| **Model**  | **NLI** |          | **Transfer** |           |
|------------|:-------:|----------|:------------:|-----------|
|            | **dev** | **test** | **micro**    | **macro** |
| LSTM       | 79.3    | 80.0     |   82.8       |  81.0     |
| LSTM+CL    | 79.9    | 79.8     | 79.5| 77.9
| BiLSTM-Last| 80.9    | 80.6     | 78.8         | 77.2      |
| BiLSTM-Last+CL  | 80.5    | 80.2 |78.5|77.1
| BiLSTM-Max | 83.5    | 83.5     |   83.6       |    82.0   |
| BiLSTM-Max+CL       | 83.6    | 83.2     |   83.5       |  82.1    |



The results show no indication that this type of curriculum learning improved the sentence representations in any way. While, accuracy has improved for the LSTM and BiLSTM + Max-Pooling on the NLI dev set, this is not the case for the other tests. The contrary actually happens for the LSTM on the transfer tasks. However, because the differences are not significant, they could also be due to randomnes in the initialisation.

More research would be necessary to analyse the influence of this type of curriculum learning, possibly through more elaborate complexity definitions. Furthermore, a different training setup could be used more suited to the curriculum learning task as the same optimizer and learning rate scheduling have been used for the non-curriculum task from the original paper, possibly ending training before converging to a better optimum.  